In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
# Fsazer um download de um dataset contendo imagens de caes e gatos.  
# O método get_file() faz o download e extrai a informação do zip: 
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/get_file
# A informação fica na pasta datasets da diretoria corrente (confirmar a divisão por pastas)

import os 

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True, cache_dir=os.curdir)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

In [ ]:
# Contar o total de imagens
import pathlib 

data_dir = pathlib.Path(PATH)

image_count = len(list(data_dir.glob('*/*/*.jpg')))
print(image_count)

In [ ]:
# Criacao dos Datasets de treino e de validação
# O método image_dataset_from_directory cria um objeto Dataset a partir de imagens armazenadas em diretorias
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory
# Permite fazer o shuffle das imagens, ajustar tamanho, definir batch size. 
# O Dataset fica praticamente pronto a ser processado pela rede

IMG_SIZE = (160, 160)

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

train_ds = image_dataset_from_directory(train_dir, shuffle=True, image_size=IMG_SIZE)


val_ds = image_dataset_from_directory(validation_dir, shuffle=True, image_size=IMG_SIZE)

In [ ]:
# Confirmar o formato dos batches do dataset

for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
# Visualizar alguns exemplos


plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8")) #pode ser transformado num array numpy 
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
# Criar um conjunto de teste retirando 33% dos exemplos da validação

val_batches = val_ds.cardinality().numpy() 
test_ds = val_ds.take(val_batches // 3) 
val_ds = val_ds.skip(val_batches // 3)

In [ ]:
# Confirmar o número de batches em cada dataset

print('Train: ', train_ds.cardinality().numpy()) 
print('Validation: ', val_ds.cardinality().numpy()) 
print('Teste: ', test_ds.cardinality().numpy())

In [ ]:
# Ajustar algumas configurações para melhorar o desempenho
# Verificar quais são as operações aplicadadas aos datasets

train_ds = train_ds.cache().prefetch(1)
val_ds = val_ds.cache().prefetch(1)

In [ ]:
# Lista e descrição dos métodos da classe Dataset

for m in dir(tf.data.Dataset):
    if not (m.startswith("_") or m.endswith("_")):
        func = getattr(tf.data.Dataset, m)
        if hasattr(func, "__doc__"):
            print("● {:21s}{}".format(m + "()", func.__doc__.split("\n")[0]))

In [ ]:
# Normalizar inputs
# Poderia ser adicionada uma camada ao início da Rede, mas aqui vai ser usado o método map

normalization_layer = keras.layers.experimental.preprocessing.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
# Confirmar que a transformação foi feita e que todos os valores variam entre 0 e 1

image_batch, labels_batch = next(iter(train_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

In [ ]:
# Que informação está guardada nos tensores image_batch e first_image
# Confirme verificando as suas dimensões



In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
# Criar uma Rede Neuronal com a seguinte arquitetura
# 4 camadas escondidas com 50 neuronios cada: Inicialização de pesos He e função ReLU
# Última camada com Sigmoid (2 classes)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[160, 160, 3]),
    keras.layers.Dense(50, kernel_initializer="he_normal", activation="relu"),   
    keras.layers.Dense(50, kernel_initializer="he_normal", activation="relu"),   
    keras.layers.Dense(50, kernel_initializer="he_normal", activation="relu"), 
    keras.layers.Dense(50, kernel_initializer="he_normal", activation="relu"), 
    keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
# Analise as características da Rede



In [ ]:
# Compilar o modelo 
# A função de loss está ajustada ao formato da saída

model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=["accuracy"])

In [ ]:
# Treinar o modelo durante 30 épocas
history = model.fit(train_ds, epochs=30, validation_data=val_ds)

In [ ]:
# Criar gráfico para visualização da evolução da accuracy
import pandas as pd

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy Evolution')
plt.legend(['train', 'val'], loc='upper left')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.grid(True)
plt.ylim(0.5, 1.0)
plt.show()

In [ ]:
# Analise os resultados, avalie o desempenho no conjunto de teste
model.evaluate(test_ds)

In [ ]:
# Teste o classificador com duas imagens à sua escolha
# Terá que ter 2 imagens na diretoria corrente e atribuir o seu nome às variáveis nome1 e nome2
# Ver os métodos keras para processamento de imagem: https://keras.io/api/preprocessing/image/

nome1 = '6.jpg'
nome2 = '5.jpg'

im1 = tf.keras.preprocessing.image.load_img(nome1, target_size=IMG_SIZE)
im2 = tf.keras.preprocessing.image.load_img(nome2, target_size=IMG_SIZE)

In [ ]:
# Transformar num array numpy, normalizar inputs e criar batch
# Não esquecer que o modelo espera um tensor de dimensão 4

input1 = keras.preprocessing.image.img_to_array(im1)
input2 = keras.preprocessing.image.img_to_array(im2)
input1 = input1.astype('float32') / 255
input2 = input2.astype('float32') / 255

images = np.array([input1, input2])
images.shape

In [ ]:
# Aplicar o modelo e obter as previsões

predictions = model.predict(images)
predictions = predictions.squeeze()
predictions = np.where(predictions > 0.5, 1,0)
print(predictions)

In [ ]:
# Mostrar o resultado

plt.figure(figsize=(10,10))
for i in range(2):
    ax = plt.subplot(2,2,i + 1)
    plt.imshow(images[i]) 
    plt.title(class_names[predictions[i]])
    plt.axis("off")

In [ ]:
# Tente melhorar o comportamento da rede, tendo em consideração diversos componentes
# 1. Arquitetura
# 2. Funções de Ativação, Inicialização de Pesos : https://www.tensorflow.org/api_docs/python/tf/keras/activations
# 3. Otimização : https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# 4. BatchNorm : https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization
# 5. Dropout : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout

# O objetivo é atingir 70% de Accuracy no conjunto de teste

# Também poderão ser usados callbacks seguinte (https://keras.io/api/callbacks/) para:
# Early Stopping se o treino estagnar e modelo entrar em overfitting
# Adcionar este callback ao parâmetro callbacks no método fit()

early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=5, restore_best_weights=True)




In [ ]:
# Utilização da Ferramenta TensorBoard

import os 

# Diretoria Raiz
root_logdir = os.path.join(os.curdir, "my_logs")

# Sub-diretoria onde ficarão os dados da execução: tem um nome criado a partir da hora de inicio
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)




In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

modelT = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[160, 160, 3]),
    keras.layers.Dense(50, kernel_initializer="he_normal", activation="relu"), 
    keras.layers.Dense(50, kernel_initializer="he_normal", activation="relu"), 
    keras.layers.Dense(1, activation="sigmoid")
])

modelT.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=["accuracy"])

# Criação do callback para executar a recolha de dados para o Tensorboard durante o treino
# Consultar os parâmetros aqui: https://keras.io/api/callbacks/tensorboard/
run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir, histogram_freq=1)


# O método fit recebe informação sobre o callback do tensorboard
history = modelT.fit(train_ds, epochs=5, validation_data=val_ds, callbacks=[tensorboard_cb])

In [ ]:
# Lançar extensão TensorBoard
%load_ext tensorboard

In [ ]:
# Iniciar o servidor TensorBoard 
# --port=6006
# --host=127.0.0.1
%tensorboard --logdir=./my_logs --host=127.0.0.1